In [ ]:
pip install ultralytics opencv-python numpy pandas matplotlib seaborn scikit-learn torch tqdm

In [ ]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import seaborn as sns
from scipy.optimize import linear_sum_assignment
from scipy.spatial.distance import cdist
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import torch
from ultralytics import YOLO
import os
from collections import defaultdict, deque
import json
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Set up plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [ ]:
class PlayerTracker:
    """
    Advanced player tracking system for cross-camera mapping
    """
    def __init__(self, max_disappeared=30, max_distance=100):
        self.next_id = 0
        self.objects = {}
        self.disappeared = {}
        self.max_disappeared = max_disappeared
        self.max_distance = max_distance
        
    def register(self, centroid, bbox, features):
        """Register a new player with unique ID"""
        self.objects[self.next_id] = {
            'centroid': centroid,
            'bbox': bbox,
            'features': features,
            'history': deque(maxlen=30)
        }
        self.disappeared[self.next_id] = 0
        self.next_id += 1
        
    def deregister(self, object_id):
        """Remove player from tracking"""
        del self.objects[object_id]
        del self.disappeared[object_id]
        
    def update(self, detections):
        """Update tracker with new detections"""
        if len(detections) == 0:
            # Mark all existing objects as disappeared
            for object_id in list(self.disappeared.keys()):
                self.disappeared[object_id] += 1
                if self.disappeared[object_id] > self.max_disappeared:
                    self.deregister(object_id)
            return
            
        # Initialize arrays for new detections
        input_centroids = []
        input_bboxes = []
        input_features = []
        
        for detection in detections:
            input_centroids.append(detection['centroid'])
            input_bboxes.append(detection['bbox'])
            input_features.append(detection['features'])
            
        # If no existing objects, register all as new
        if len(self.objects) == 0:
            for i in range(len(input_centroids)):
                self.register(input_centroids[i], input_bboxes[i], input_features[i])
        else:
            # Match existing objects to new detections
            object_centroids = [obj['centroid'] for obj in self.objects.values()]
            object_ids = list(self.objects.keys())
            
            # Compute distance matrix
            D = cdist(np.array(object_centroids), np.array(input_centroids))
            
            # Hungarian algorithm for optimal assignment
            row_ind, col_ind = linear_sum_assignment(D)
            
            used_row_indices = set()
            used_col_indices = set()
            
            # Update matched objects
            for (row, col) in zip(row_ind, col_ind):
                if D[row, col] <= self.max_distance:
                    object_id = object_ids[row]
                    self.objects[object_id]['centroid'] = input_centroids[col]
                    self.objects[object_id]['bbox'] = input_bboxes[col]
                    self.objects[object_id]['features'] = input_features[col]
                    self.objects[object_id]['history'].append(input_centroids[col])
                    self.disappeared[object_id] = 0
                    
                    used_row_indices.add(row)
                    used_col_indices.add(col)
                    
            # Handle unmatched detections and objects
            unused_row_indices = set(range(0, D.shape[0])).difference(used_row_indices)
            unused_col_indices = set(range(0, D.shape[1])).difference(used_col_indices)
            
            # Mark unmatched existing objects as disappeared
            for row in unused_row_indices:
                object_id = object_ids[row]
                self.disappeared[object_id] += 1
                if self.disappeared[object_id] > self.max_disappeared:
                    self.deregister(object_id)
                    
            # Register new objects
            for col in unused_col_indices:
                self.register(input_centroids[col], input_bboxes[col], input_features[col])

In [ ]:
class CrossCameraMapper:
    """
    Main class for mapping players across different camera views
    """
    def __init__(self, model_path):
        self.model = YOLO(model_path)
        self.broadcast_tracker = PlayerTracker()
        self.tacticam_tracker = PlayerTracker()
        self.player_mappings = {}
        
    def extract_features(self, frame, bbox):
        """Extract visual features from player bounding box"""
        x1, y1, x2, y2 = map(int, bbox)
        
        # Ensure coordinates are within frame bounds
        h, w = frame.shape[:2]
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(w, x2), min(h, y2)
        
        if x2 <= x1 or y2 <= y1:
            return np.zeros(10)  # Return zero features for invalid bbox
            
        player_roi = frame[y1:y2, x1:x2]
        
        if player_roi.size == 0:
            return np.zeros(10)
            
        # Color histogram features
        hist_b = cv2.calcHist([player_roi], [0], None, [32], [0, 256])
        hist_g = cv2.calcHist([player_roi], [1], None, [32], [0, 256])
        hist_r = cv2.calcHist([player_roi], [2], None, [32], [0, 256])
        
        # Normalize histograms
        hist_b = hist_b.flatten() / (hist_b.sum() + 1e-6)
        hist_g = hist_g.flatten() / (hist_g.sum() + 1e-6)
        hist_r = hist_r.flatten() / (hist_r.sum() + 1e-6)
        
        # Spatial features
        aspect_ratio = (x2 - x1) / (y2 - y1) if (y2 - y1) > 0 else 1.0
        area = (x2 - x1) * (y2 - y1)
        
        # Combine features
        features = np.concatenate([
            hist_b[:8],  # Reduced histogram size
            hist_g[:8],
            hist_r[:8],
            [aspect_ratio, area / (frame.shape[0] * frame.shape[1])]  # Normalized area
        ])
        
        return features[:10]  # Ensure fixed feature size
        
    def detect_players(self, frame, debug=False):
        """Detect players in frame using YOLO model with proper class filtering"""
        results = self.model(frame, verbose=False)
        detections = []
        
        # Debug: Print all detected classes and their names
        if debug and hasattr(self.model, 'names'):
            print(f"Model classes: {self.model.names}")
        
        for result in results:
            boxes = result.boxes
            if boxes is not None:
                if debug:
                    print(f"Found {len(boxes)} total detections")
                
                for i, box in enumerate(boxes):
                    # Get box coordinates and confidence
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                    conf = box.conf[0].cpu().numpy()
                    cls = int(box.cls[0].cpu().numpy())
                    
                    if debug:
                        class_name = self.model.names.get(cls, f"class_{cls}") if hasattr(self.model, 'names') else f"class_{cls}"
                        print(f"Detection {i}: class={cls} ({class_name}), conf={conf:.3f}, bbox=({x1:.1f},{y1:.1f},{x2:.1f},{y2:.1f})")
                    
                    # Filter for player classes - try multiple possible class indices
                    # Common class indices for players: 0 (person), 1 (player), etc.
                    player_classes = [1,2,3]  # Add more class indices if needed
                    ball_classes = [0]  # Common ball class indices (ball, sports ball, etc.)
                    
                    # Check if this is a player class (not ball)
                    is_player = cls in player_classes or (cls not in ball_classes and conf > 0.3)
                    
                    # Also filter by size - players should be reasonably sized
                    width = x2 - x1
                    height = y2 - y1
                    area = width * height
                    aspect_ratio = height / width if width > 0 else 0
                    
                    # Player size filters (adjust based on your video resolution)
                    min_area = 500  # Minimum pixel area for a player
                    max_area = frame.shape[0] * frame.shape[1] * 0.3  # Max 30% of frame
                    min_aspect_ratio = 1.2  # Players are usually taller than wide
                    max_aspect_ratio = 4.0  # But not too tall
                    
                    size_filter = (min_area < area < max_area and 
                                 min_aspect_ratio < aspect_ratio < max_aspect_ratio)
                    
                    if conf > 0.3 and is_player and size_filter:
                        centroid = ((x1 + x2) / 2, (y1 + y2) / 2)
                        bbox = (x1, y1, x2, y2)
                        features = self.extract_features(frame, bbox)
                        
                        detections.append({
                            'centroid': centroid,
                            'bbox': bbox,
                            'features': features,
                            'confidence': conf,
                            'class': cls,
                            'area': area,
                            'aspect_ratio': aspect_ratio
                        })
                        
                        if debug:
                            print(f"✓ Accepted as player: class={cls}, conf={conf:.3f}, area={area:.0f}, aspect_ratio={aspect_ratio:.2f}")
                    elif debug:
                        reasons = []
                        if conf <= 0.3:
                            reasons.append(f"low_conf({conf:.3f})")
                        if not is_player:
                            reasons.append(f"not_player_class({cls})")
                        if not size_filter:
                            reasons.append(f"size_filter(area={area:.0f},ar={aspect_ratio:.2f})")
                        print(f"✗ Rejected: {', '.join(reasons)}")
        
        if debug:
            print(f"Final detections: {len(detections)} players")
            
        return detections
    
    def process_video(self, video_path, tracker, max_frames=None, visualize=True, output_path=None, debug_first_frame=True):
        """Process video and track players with optional visualization and debugging"""
        cap = cv2.VideoCapture(video_path)
        frame_count = 0
        results = []
        
        # Get video properties
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        if max_frames:
            total_frames = min(total_frames, max_frames)
        
        # Setup video writer for output if requested
        video_writer = None
        if visualize and output_path:
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            video_writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
            
        pbar = tqdm(total=total_frames, desc=f"Processing {os.path.basename(video_path)}")
        
        # Define colors for different player IDs
        colors = [
            (255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255),
            (0, 255, 255), (128, 0, 128), (255, 165, 0), (0, 128, 128), (128, 128, 0),
            (255, 192, 203), (0, 255, 127), (255, 20, 147), (32, 178, 170), (255, 69, 0)
        ]
        
        while True:
            ret, frame = cap.read()
            if not ret or (max_frames and frame_count >= max_frames):
                break
            
            # Debug first frame to understand what's being detected
            debug_mode = debug_first_frame and frame_count == 0
            if debug_mode:
                print(f"\n=== DEBUGGING FIRST FRAME of {os.path.basename(video_path)} ===")
                
            # Detect players
            detections = self.detect_players(frame, debug=debug_mode)
            
            if debug_mode:
                print(f"=== END DEBUG FIRST FRAME ===\n")
                
            # Update tracker
            tracker.update(detections)
            
            # Create visualization frame
            vis_frame = frame.copy()
            
            # Draw all raw detections first (for debugging)
            if visualize:
                # Draw raw YOLO detections in light blue
                raw_results = self.model(frame, verbose=False)
                for result in raw_results:
                    boxes = result.boxes
                    if boxes is not None:
                        for box in boxes:
                            x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())
                            conf = box.conf[0].cpu().numpy()
                            cls = int(box.cls[0].cpu().numpy())
                            
                            # Draw all detections in light blue
                            cv2.rectangle(vis_frame, (x1, y1), (x2, y2), (173, 216, 230), 1)
                            
                            # Label with class and confidence
                            class_name = self.model.names.get(cls, f"c{cls}") if hasattr(self.model, 'names') else f"c{cls}"
                            label = f"{class_name}:{conf:.2f}"
                            cv2.putText(vis_frame, label, (x1, y1-5), 
                                       cv2.FONT_HERSHEY_SIMPLEX, 0.4, (173, 216, 230), 1)
            
            # Draw tracked players (filtered detections) in bright colors
            for player_id, player_data in tracker.objects.items():
                x1, y1, x2, y2 = map(int, player_data['bbox'])
                centroid = player_data['centroid']
                
                # Get color for this player ID
                color = colors[player_id % len(colors)]
                
                # Draw bounding box (thick)
                cv2.rectangle(vis_frame, (x1, y1), (x2, y2), color, 3)
                
                # Draw centroid
                cv2.circle(vis_frame, (int(centroid[0]), int(centroid[1])), 5, color, -1)
                
                # Draw player ID
                label = f"Player {player_id}"
                label_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)[0]
                cv2.rectangle(vis_frame, (x1, y1 - label_size[1] - 10), 
                             (x1 + label_size[0], y1), color, -1)
                cv2.putText(vis_frame, label, (x1, y1 - 5), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                
                # Draw trajectory (if available)
                if len(player_data['history']) > 1:
                    points = np.array(player_data['history'], dtype=np.int32)
                    cv2.polylines(vis_frame, [points], False, color, 2)
            
            # Add comprehensive frame info
            info_lines = [
                f"Frame: {frame_count} | Tracked Players: {len(tracker.objects)}",
                f"Raw Detections: {len(detections)} | Video: {os.path.basename(video_path)}"
            ]
            
            for i, info_text in enumerate(info_lines):
                cv2.putText(vis_frame, info_text, (10, 30 + i*25), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            
            # Add legend
            legend_y = height - 60
            cv2.putText(vis_frame, "Light Blue: All YOLO detections", (10, legend_y), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (173, 216, 230), 2)
            cv2.putText(vis_frame, "Bright Colors: Tracked Players", (10, legend_y + 20), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            
            # Display frame if visualizing
            if visualize:
                # Resize for display if too large
                display_frame = vis_frame
                if width > 1280:
                    scale = 1280 / width
                    new_width = int(width * scale)
                    new_height = int(height * scale)
                    display_frame = cv2.resize(vis_frame, (new_width, new_height))
                
                cv2.imshow(f'Tracking - {os.path.basename(video_path)}', display_frame)
                
                # Press 'q' to quit, 'p' to pause, 'd' for debug info
                key = cv2.waitKey(1) & 0xFF
                if key == ord('q'):
                    break
                elif key == ord('p'):
                    cv2.waitKey(0)  # Wait for any key to continue
                elif key == ord('d'):
                    # Print debug info for current frame
                    print(f"\n=== DEBUG INFO Frame {frame_count} ===")
                    debug_detections = self.detect_players(frame, debug=True)
                    print(f"=== END DEBUG INFO ===\n")
            
            # Write frame to output video
            if video_writer:
                video_writer.write(vis_frame)
            
            # Store results
            frame_data = {
                'frame': frame_count,
                'players': {}
            }
            
            for player_id, player_data in tracker.objects.items():
                frame_data['players'][player_id] = {
                    'centroid': player_data['centroid'],
                    'bbox': player_data['bbox'],
                    'features': player_data['features']
                }
                
            results.append(frame_data)
            frame_count += 1
            pbar.update(1)
            
        cap.release()
        if video_writer:
            video_writer.release()
        cv2.destroyAllWindows()
        pbar.close()
        return results
    
    def compute_feature_similarity(self, features1, features2):
        """Compute similarity between feature vectors"""
        # Handle zero features
        if np.all(features1 == 0) or np.all(features2 == 0):
            return 0.0
            
        # Cosine similarity
        dot_product = np.dot(features1, features2)
        norm1 = np.linalg.norm(features1)
        norm2 = np.linalg.norm(features2)
        
        if norm1 == 0 or norm2 == 0:
            return 0.0
            
        return dot_product / (norm1 * norm2)
    
    def map_players_across_cameras(self, broadcast_results, tacticam_results):
        """Map players between broadcast and tacticam views"""
        print("Mapping players across cameras...")
        
        # Collect all player features from both cameras
        broadcast_features = defaultdict(list)
        tacticam_features = defaultdict(list)
        
        # Extract features for each player across all frames
        for frame_data in broadcast_results:
            for player_id, player_data in frame_data['players'].items():
                broadcast_features[player_id].append(player_data['features'])
                
        for frame_data in tacticam_results:
            for player_id, player_data in frame_data['players'].items():
                tacticam_features[player_id].append(player_data['features'])
        
        # Compute average features for each player
        broadcast_avg_features = {}
        for player_id, features_list in broadcast_features.items():
            if features_list:
                broadcast_avg_features[player_id] = np.mean(features_list, axis=0)
        
        tacticam_avg_features = {}
        for player_id, features_list in tacticam_features.items():
            if features_list:
                tacticam_avg_features[player_id] = np.mean(features_list, axis=0)
        
        # Create similarity matrix
        broadcast_ids = list(broadcast_avg_features.keys())
        tacticam_ids = list(tacticam_avg_features.keys())
        
        if not broadcast_ids or not tacticam_ids:
            print("Warning: No players found in one or both videos")
            return {}
        
        similarity_matrix = np.zeros((len(broadcast_ids), len(tacticam_ids)))
        
        for i, b_id in enumerate(broadcast_ids):
            for j, t_id in enumerate(tacticam_ids):
                similarity = self.compute_feature_similarity(
                    broadcast_avg_features[b_id],
                    tacticam_avg_features[t_id]
                )
                similarity_matrix[i, j] = similarity
        
        # Use Hungarian algorithm to find optimal mapping
        # Convert similarity to cost (distance)
        cost_matrix = 1 - similarity_matrix
        row_ind, col_ind = linear_sum_assignment(cost_matrix)
        
        # Create mappings
        mappings = {}
        for i, j in zip(row_ind, col_ind):
            if similarity_matrix[i, j] > 0.3:  # Threshold for valid mapping
                broadcast_id = broadcast_ids[i]
                tacticam_id = tacticam_ids[j]
                mappings[tacticam_id] = broadcast_id
                
        print(f"Successfully mapped {len(mappings)} players")
        return mappings, similarity_matrix, broadcast_ids, tacticam_ids
    
    def visualize_mappings(self, similarity_matrix, broadcast_ids, tacticam_ids, mappings):
        """Visualize player mappings"""
        plt.figure(figsize=(12, 8))
        
        # Plot similarity matrix
        plt.subplot(2, 2, 1)
        sns.heatmap(similarity_matrix, 
                   xticklabels=[f'Tacticam_{id}' for id in tacticam_ids],
                   yticklabels=[f'Broadcast_{id}' for id in broadcast_ids],
                   annot=True, fmt='.2f', cmap='viridis')
        plt.title('Player Similarity Matrix')
        plt.xlabel('Tacticam Players')
        plt.ylabel('Broadcast Players')
        
        # Plot mapping results
        plt.subplot(2, 2, 2)
        mapping_data = []
        for tacticam_id, broadcast_id in mappings.items():
            mapping_data.append({
                'Tacticam_ID': tacticam_id,
                'Broadcast_ID': broadcast_id,
                'Similarity': similarity_matrix[broadcast_ids.index(broadcast_id), 
                                              tacticam_ids.index(tacticam_id)]
            })
        
        if mapping_data:
            df = pd.DataFrame(mapping_data)
            bars = plt.bar(range(len(df)), df['Similarity'])
            plt.xlabel('Player Pairs')
            plt.ylabel('Similarity Score')
            plt.title('Player Mapping Confidence')
            plt.xticks(range(len(df)), 
                      [f'T{row.Tacticam_ID}→B{row.Broadcast_ID}' for _, row in df.iterrows()],
                      rotation=45)
            
            # Color bars based on confidence
            for bar, sim in zip(bars, df['Similarity']):
                if sim > 0.7:
                    bar.set_color('green')
                elif sim > 0.5:
                    bar.set_color('orange')
                else:
                    bar.set_color('red')
        
        plt.tight_layout()
        plt.show()
        
        return mapping_data if mapping_data else []
    
    def apply_consistent_ids(self, tacticam_results, mappings):
        """Apply consistent player IDs to tacticam results"""
        print("Applying consistent player IDs...")
        
        consistent_results = []
        for frame_data in tacticam_results:
            new_frame_data = {
                'frame': frame_data['frame'],
                'players': {}
            }
            
            for tacticam_id, player_data in frame_data['players'].items():
                # Get consistent ID from mapping
                if tacticam_id in mappings:
                    consistent_id = mappings[tacticam_id]
                else:
                    consistent_id = f"unmapped_{tacticam_id}"
                
                new_frame_data['players'][consistent_id] = player_data
                
            consistent_results.append(new_frame_data)
            
    def create_side_by_side_visualization(self, broadcast_video, tacticam_video, mappings, max_frames=300, output_path="mapped_tracking.mp4"):
        """Create side-by-side video showing mapped players across both cameras"""
        print("Creating side-by-side visualization with mapped players...")
        
        # Open both videos
        cap1 = cv2.VideoCapture(broadcast_video)
        cap2 = cv2.VideoCapture(tacticam_video)
        
        # Get video properties
        fps1 = int(cap1.get(cv2.CAP_PROP_FPS))
        fps2 = int(cap2.get(cv2.CAP_PROP_FPS))
        fps = min(fps1, fps2)  # Use minimum fps
        
        width1 = int(cap1.get(cv2.CAP_PROP_FRAME_WIDTH))
        height1 = int(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT))
        width2 = int(cap2.get(cv2.CAP_PROP_FRAME_WIDTH))
        height2 = int(cap2.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
        # Make both frames same height
        target_height = min(height1, height2)
        scale1 = target_height / height1
        scale2 = target_height / height2
        new_width1 = int(width1 * scale1)
        new_width2 = int(width2 * scale2)
        
        # Setup video writer
        output_width = new_width1 + new_width2
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        video_writer = cv2.VideoWriter(output_path, fourcc, fps, (output_width, target_height))
        
        # Reset trackers
        self.broadcast_tracker = PlayerTracker()
        self.tacticam_tracker = PlayerTracker()
        
        # Define consistent colors for mapped players
        mapped_colors = {}
        base_colors = [
            (255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255),
            (0, 255, 255), (128, 0, 128), (255, 165, 0), (0, 128, 128), (128, 128, 0),
            (255, 192, 203), (0, 255, 127), (255, 20, 147), (32, 178, 170), (255, 69, 0)
        ]
        
        frame_count = 0
        pbar = tqdm(total=max_frames, desc="Creating side-by-side video")
        
        while frame_count < max_frames:
            ret1, frame1 = cap1.read()
            ret2, frame2 = cap2.read()
            
            if not ret1 or not ret2:
                break
            
            # Resize frames
            frame1 = cv2.resize(frame1, (new_width1, target_height))
            frame2 = cv2.resize(frame2, (new_width2, target_height))
            
            # Process both frames
            detections1 = self.detect_players(frame1)
            detections2 = self.detect_players(frame2)
            
            self.broadcast_tracker.update(detections1)
            self.tacticam_tracker.update(detections2)
            
            # Draw tracking on broadcast frame
            for player_id, player_data in self.broadcast_tracker.objects.items():
                x1, y1, x2, y2 = map(int, player_data['bbox'])
                centroid = player_data['centroid']
                
                # Use consistent color
                if player_id not in mapped_colors:
                    mapped_colors[player_id] = base_colors[len(mapped_colors) % len(base_colors)]
                color = mapped_colors[player_id]
                
                # Draw bounding box and ID
                cv2.rectangle(frame1, (x1, y1), (x2, y2), color, 2)
                cv2.circle(frame1, (int(centroid[0]), int(centroid[1])), 5, color, -1)
                
                label = f"B{player_id}"
                cv2.rectangle(frame1, (x1, y1 - 25), (x1 + 50, y1), color, -1)
                cv2.putText(frame1, label, (x1 + 5, y1 - 5), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            
            # Draw tracking on tacticam frame
            for player_id, player_data in self.tacticam_tracker.objects.items():
                x1, y1, x2, y2 = map(int, player_data['bbox'])
                centroid = player_data['centroid']
                
                # Check if this player is mapped
                mapped_id = mappings.get(player_id, None)
                if mapped_id is not None and mapped_id in mapped_colors:
                    color = mapped_colors[mapped_id]
                    label = f"T{player_id}→B{mapped_id}"
                else:
                    color = (128, 128, 128)  # Gray for unmapped
                    label = f"T{player_id}"
                
                # Draw bounding box and ID
                cv2.rectangle(frame2, (x1, y1), (x2, y2), color, 2)
                cv2.circle(frame2, (int(centroid[0]), int(centroid[1])), 5, color, -1)
                
                label_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]
                cv2.rectangle(frame2, (x1, y1 - 25), (x1 + label_size[0] + 10, y1), color, -1)
                cv2.putText(frame2, label, (x1 + 5, y1 - 5), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            
            # Add titles
            cv2.putText(frame1, "BROADCAST VIEW", (10, 30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            cv2.putText(frame2, "TACTICAM VIEW", (10, 30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            
            # Add frame counter
            cv2.putText(frame1, f"Frame: {frame_count}", (10, target_height - 20), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            cv2.putText(frame2, f"Mapped: {len(mappings)} players", (10, target_height - 20), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            
            # Combine frames side by side
            combined_frame = np.hstack([frame1, frame2])
            
            # Write to output video
            video_writer.write(combined_frame)
            
            frame_count += 1
            pbar.update(1)
        
        # Cleanup
        cap1.release()
        cap2.release()
        video_writer.release()
        pbar.close()
        
        print(f"✓ Side-by-side video saved as '{output_path}'")
        return output_path

In [ ]:
# Main execution function
def main():
    """
    Main function to execute cross-camera player mapping
    """
    print("=== Cross-Camera Player Mapping System ===\n")
    
    # Configuration
    MODEL_PATH = "best.pt"  # Update with actual model path
    BROADCAST_VIDEO = "broadcast.mp4"
    TACTICAM_VIDEO = "tacticam.mp4"
    MAX_FRAMES = 300  # Process first 300 frames for demonstration
    
    # Initialize mapper
    print("1. Initializing Cross-Camera Mapper...")
    try:
        mapper = CrossCameraMapper(MODEL_PATH)
        print("✓ Model loaded successfully")
    except Exception as e:
        print(f"✗ Failed to load model: {e}")
        print("Please download the model from the provided Google Drive link")
        return
    
    # Process videos with visualization
    print("\n2. Processing Videos with Live Tracking Visualization...")
    try:
        print("Processing broadcast video...")
        print("Controls: Press 'q' to quit, 'p' to pause")
        broadcast_results = mapper.process_video(BROADCAST_VIDEO, 
                                                mapper.broadcast_tracker, 
                                                MAX_FRAMES,
                                                visualize=True,
                                                output_path="broadcast_tracked.mp4")
        
        print("Processing tacticam video...")
        print("Controls: Press 'q' to quit, 'p' to pause")
        tacticam_results = mapper.process_video(TACTICAM_VIDEO, 
                                               mapper.tacticam_tracker, 
                                               MAX_FRAMES,
                                               visualize=True,
                                               output_path="tacticam_tracked.mp4")
        
        print("✓ Video processing completed")
        print("✓ Tracked videos saved as 'broadcast_tracked.mp4' and 'tacticam_tracked.mp4'")
    except Exception as e:
        print(f"✗ Failed to process videos: {e}")
        return
    
    # Map players across cameras
    print("\n3. Mapping Players Across Cameras...")
    try:
        mappings, similarity_matrix, broadcast_ids, tacticam_ids = mapper.map_players_across_cameras(
            broadcast_results, tacticam_results)
        
        print("✓ Player mapping completed")
        print(f"Found {len(mappings)} player correspondences")
        
        # Display mappings
        print("\nPlayer Mappings:")
        for tacticam_id, broadcast_id in mappings.items():
            print(f"  Tacticam Player {tacticam_id} → Broadcast Player {broadcast_id}")
            
    except Exception as e:
        print(f"✗ Failed to map players: {e}")
        return
    
    # Visualize results
    print("\n4. Visualizing Results...")
    try:
        mapping_data = mapper.visualize_mappings(similarity_matrix, broadcast_ids, 
                                                tacticam_ids, mappings)
        
        # Apply consistent IDs
        consistent_tacticam_results = mapper.apply_consistent_ids(tacticam_results, mappings)
        
        print("✓ Visualization completed")
        
        # Create side-by-side comparison video
        print("\n4.1. Creating Side-by-Side Mapped Video...")
        try:
            side_by_side_path = mapper.create_side_by_side_visualization(
                BROADCAST_VIDEO, TACTICAM_VIDEO, mappings, MAX_FRAMES)
            print("✓ Side-by-side video created successfully")
        except Exception as e:
            print(f"✗ Failed to create side-by-side video: {e}")
        
    except Exception as e:
        print(f"✗ Failed to visualize results: {e}")
        return
    
    # Save results
    print("\n5. Saving Results...")
    try:
        # Save mappings to JSON
        output_data = {
            'mappings': mappings,
            'mapping_confidence': mapping_data,
            'total_frames_processed': len(broadcast_results),
            'broadcast_players': len(set(broadcast_ids)),
            'tacticam_players': len(set(tacticam_ids))
        }
        
        with open('player_mappings.json', 'w') as f:
            json.dump(output_data, f, indent=2, default=str)
            
        print("✓ Results saved to 'player_mappings.json'")
        
        # Print summary statistics
        print(f"\n=== Summary ===")
        print(f"Frames processed: {len(broadcast_results)}")
        print(f"Broadcast players detected: {len(broadcast_ids)}")
        print(f"Tacticam players detected: {len(tacticam_ids)}")
        print(f"Successful mappings: {len(mappings)}")
        
        if mapping_data:
            avg_confidence = np.mean([item['Similarity'] for item in mapping_data])
            print(f"Average mapping confidence: {avg_confidence:.3f}")
        
    except Exception as e:
        print(f"✗ Failed to save results: {e}")
    
    print("\n=== Cross-Camera Player Mapping Complete ===")

In [ ]:
def run_with_visualization():
    """
    Enhanced version with real-time visualization and video output
    """
    print("=== Cross-Camera Player Mapping with Live Visualization ===\n")
    
    # Configuration
    MODEL_PATH = "best.pt"  # Update with actual model path
    BROADCAST_VIDEO = "broadcast.mp4"
    TACTICAM_VIDEO = "tacticam.mp4"
    MAX_FRAMES = 300  # Process first 300 frames for demonstration
    
    # Initialize mapper
    print("1. Initializing Cross-Camera Mapper...")
    try:
        mapper = CrossCameraMapper(MODEL_PATH)
        print("✓ Model loaded successfully")
    except Exception as e:
        print(f"✗ Failed to load model: {e}")
        print("Please download the model from the provided Google Drive link")
        return
    
    # Process videos with real-time visualization
    print("\n2. Processing Videos with Live Tracking...")
    print("📺 Live tracking windows will open - use 'q' to quit, 'p' to pause")
    print("🎥 Output videos will be saved automatically")
    
    try:
        # Process broadcast video with live visualization
        print("\nProcessing broadcast video...")
        broadcast_results = mapper.process_video(
            BROADCAST_VIDEO, 
            mapper.broadcast_tracker, 
            MAX_FRAMES,
            visualize=True,
            output_path="broadcast_tracked.mp4"
        )
        
        # Reset tracker for tacticam
        mapper.tacticam_tracker = PlayerTracker()
        
        # Process tacticam video with live visualization
        print("\nProcessing tacticam video...")
        tacticam_results = mapper.process_video(
            TACTICAM_VIDEO, 
            mapper.tacticam_tracker, 
            MAX_FRAMES,
            visualize=True,
            output_path="tacticam_tracked.mp4"
        )
        
        print("✓ Individual tracking videos saved")
        
    except Exception as e:
        print(f"✗ Failed to process videos: {e}")
        return
    
    # Map players across cameras
    print("\n3. Mapping Players Across Cameras...")
    try:
        mappings, similarity_matrix, broadcast_ids, tacticam_ids = mapper.map_players_across_cameras(
            broadcast_results, tacticam_results)
        
        print("✓ Player mapping completed")
        print(f"Found {len(mappings)} player correspondences")
        
        # Display mappings
        print("\nPlayer Mappings:")
        for tacticam_id, broadcast_id in mappings.items():
            print(f"  Tacticam Player {tacticam_id} → Broadcast Player {broadcast_id}")
            
    except Exception as e:
        print(f"✗ Failed to map players: {e}")
        return
    
    # Create visualizations
    print("\n4. Creating Final Visualizations...")
    try:
        mapping_data = mapper.visualize_mappings(similarity_matrix, broadcast_ids, 
                                                tacticam_ids, mappings)
        
        # Create side-by-side comparison video
        print("\n4.1. Creating Side-by-Side Mapped Video...")
        side_by_side_path = mapper.create_side_by_side_visualization(
            BROADCAST_VIDEO, TACTICAM_VIDEO, mappings, MAX_FRAMES)
        
        print("✓ All visualizations completed")
        
    except Exception as e:
        print(f"✗ Failed to create visualizations: {e}")
        return
    
    # Save results
    print("\n5. Saving Results...")
    try:
        output_data = {
            'mappings': mappings,
            'mapping_confidence': mapping_data,
            'total_frames_processed': len(broadcast_results),
            'broadcast_players': len(set(broadcast_ids)),
            'tacticam_players': len(set(tacticam_ids)),
            'output_files': {
                'broadcast_tracked': 'broadcast_tracked.mp4',
                'tacticam_tracked': 'tacticam_tracked.mp4',
                'side_by_side': side_by_side_path
            }
        }
        
        with open('player_mappings_with_videos.json', 'w') as f:
            json.dump(output_data, f, indent=2, default=str)
            
        print("✓ Results saved to 'player_mappings_with_videos.json'")
        
        # Print summary
        print(f"\n=== Output Files Created ===")
        print(f"📹 Individual tracked videos:")
        print(f"  - broadcast_tracked.mp4 (Broadcast view with player tracking)")
        print(f"  - tacticam_tracked.mp4 (Tacticam view with player tracking)")
        print(f"📺 Combined visualization:")
        print(f"  - {side_by_side_path} (Side-by-side with mapped players)")
        print(f"📊 Analysis results:")
        print(f"  - player_mappings_with_videos.json (Detailed mapping data)")
        
        print(f"\n=== Summary Statistics ===")
        print(f"Frames processed: {len(broadcast_results)}")
        print(f"Broadcast players detected: {len(broadcast_ids)}")
        print(f"Tacticam players detected: {len(tacticam_ids)}")
        print(f"Successful mappings: {len(mappings)}")
        
        if mapping_data:
            avg_confidence = np.mean([item['Similarity'] for item in mapping_data])
            print(f"Average mapping confidence: {avg_confidence:.3f}")
        
    except Exception as e:
        print(f"✗ Failed to save results: {e}")
    
    print("\n=== Cross-Camera Player Mapping with Visualization Complete ===")

In [ ]:
def quick_test_tracking():
    """
    Quick test function to verify tracking on a single video
    """
    print("=== Quick Tracking Test ===\n")
    
    MODEL_PATH = "best.pt"
    TEST_VIDEO = "broadcast.mp4"  # Change to your test video
    
    try:
        mapper = CrossCameraMapper(MODEL_PATH)
        print("✓ Model loaded successfully")
        
        print(f"\n🎥 Testing tracking on {TEST_VIDEO}")
        print("Press 'q' to quit, 'p' to pause during playback")
        
        results = mapper.process_video(
            TEST_VIDEO,
            PlayerTracker(),
            max_frames=100,  # Test with 100 frames
            visualize=True,
            output_path="test_tracking.mp4"
        )
        
        print(f"✓ Test completed! Output saved as 'test_tracking.mp4'")
        print(f"Processed {len(results)} frames")
        
    except Exception as e:
        print(f"✗ Test failed: {e}")

In [ ]:
def debug_model_classes():
    """
    Debug function to understand what the YOLO model detects
    """
    print("=== YOLO Model Class Debugging ===\n")
    
    MODEL_PATH = "best.pt"
    TEST_VIDEO = "broadcast.mp4"  # Change to your test video
    
    try:
        # Load model
        model = YOLO(MODEL_PATH)
        print("✓ Model loaded successfully")
        
        # Print model information
        if hasattr(model, 'names'):
            print(f"\nModel classes: {model.names}")
            print(f"Number of classes: {len(model.names)}")
        else:
            print("Warning: Model doesn't have class names attribute")
        
        # Test on first frame
        cap = cv2.VideoCapture(TEST_VIDEO)
        ret, frame = cap.read()
        
        if not ret:
            print(f"✗ Could not read from {TEST_VIDEO}")
            return
            
        print(f"\nTesting on first frame of {TEST_VIDEO}")
        print(f"Frame shape: {frame.shape}")
        
        # Run inference
        results = model(frame, verbose=False)
        
        print(f"\nAll detections found:")
        detection_count = {}
        
        for result in results:
            boxes = result.boxes
            if boxes is not None:
                print(f"Total detections: {len(boxes)}")
                
                for i, box in enumerate(boxes):
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                    conf = box.conf[0].cpu().numpy()
                    cls = int(box.cls[0].cpu().numpy())
                    
                    # Get class name
                    class_name = model.names.get(cls, f"unknown_class_{cls}") if hasattr(model, 'names') else f"class_{cls}"
                    
                    # Calculate size info
                    width = x2 - x1
                    height = y2 - y1
                    area = width * height
                    aspect_ratio = height / width if width > 0 else 0
                    
                    print(f"  {i+1}. Class: {cls} ({class_name})")
                    print(f"      Confidence: {conf:.3f}")
                    print(f"      BBox: ({x1:.1f}, {y1:.1f}, {x2:.1f}, {y2:.1f})")
                    print(f"      Size: {width:.1f}x{height:.1f} (area={area:.0f})")
                    print(f"      Aspect ratio: {aspect_ratio:.2f}")
                    print()
                    
                    # Count detections by class
                    if class_name not in detection_count:
                        detection_count[class_name] = 0
                    detection_count[class_name] += 1
            else:
                print("No detections found!")
        
        print(f"Detection summary:")
        for class_name, count in detection_count.items():
            print(f"  {class_name}: {count} detections")
            
        cap.release()
        
        # Recommendations
        print(f"\n=== RECOMMENDATIONS ===")
        print("Based on the detections above, you should:")
        print("1. Identify which class represents 'players' (usually 'person' or 'player')")
        print("2. Identify which class represents 'ball' (usually 'ball' or 'sports ball')")
        print("3. Update the player_classes and ball_classes lists in detect_players()")
        print("\nExample fixes:")
        if 'person' in detection_count:
            print("  - If 'person' are your players, set player_classes = [class_id_for_person]")
        if 'ball' in detection_count:
            print("  - If 'ball' is the ball, add its class_id to ball_classes")
        
    except Exception as e:
        print(f"✗ Debug failed: {e}")

In [ ]:
def quick_test_with_debug():
    """
    Quick test with comprehensive debugging
    """
    print("=== Quick Test with Debug Mode ===\n")
    
    MODEL_PATH = "best.pt"
    TEST_VIDEO = "broadcast.mp4"  # Change to your test video
    
    try:
        mapper = CrossCameraMapper(MODEL_PATH)
        print("✓ Model loaded successfully")
        
        print(f"\n🎥 Testing tracking on {TEST_VIDEO} with debug mode")
        print("Controls:")
        print("  - Press 'q' to quit")
        print("  - Press 'p' to pause") 
        print("  - Press 'd' to show debug info for current frame")
        print("\nThe video will show:")
        print("  - Light blue boxes: All YOLO detections")
        print("  - Bright colored boxes: Successfully tracked players")
        
        results = mapper.process_video(
            TEST_VIDEO,
            PlayerTracker(),
            max_frames=50,  # Test with 50 frames
            visualize=True,
            output_path="debug_tracking.mp4",
            debug_first_frame=True  # This will show detailed debug info for first frame
        )
        
        print(f"\n✓ Test completed!")
        print(f"Processed {len(results)} frames")
        
        # Count players found
        total_players = set()
        for frame_data in results:
            total_players.update(frame_data['players'].keys())
        
        print(f"Total unique players tracked: {len(total_players)}")
        if len(total_players) == 0:
            print("\n❌ NO PLAYERS TRACKED!")
            print("This means the class filtering is wrong.")
            print("Run debug_model_classes() to see what the model actually detects.")
        else:
            print(f"✅ Successfully tracked players: {list(total_players)}")
        
    except Exception as e:
        print(f"✗ Test failed: {e}")

In [ ]:
debug_model_classes()

In [ ]:
quick_test_with_debug()

In [ ]:
quick_test_tracking()

In [ ]:
run_with_visualization()